# Opentunes API Node (Google Colab)
Run this notebook to host the GPU Backend for free.

In [ ]:
# 1. Mount Google Drive (Source of Code & Models)
from google.colab import drive
drive.mount('/content/drive')

# --- CONFIG: YOUR DRIVE PATH ---
BASE_PATH = '/content/drive/MyDrive/AI Projects/Opentunes AI'
# -------------------------------

import os
if not os.path.exists(BASE_PATH):
    print(f"⚠️ Warning: Folder '{BASE_PATH}' not found. Creating it...")
    os.makedirs(os.path.join(BASE_PATH, 'code'), exist_ok=True)
    os.makedirs(os.path.join(BASE_PATH, 'models'), exist_ok=True)
    print(f"Please upload 'opentunes.zip' to '{BASE_PATH}/code' folder!")

In [ ]:
# 2. Install Dependencies & Code
!apt-get install -y ffmpeg

# Create Workspace
!mkdir -p /content/workspace
%cd /content/workspace

# Unzip Code
import os
BASE_PATH = '/content/drive/MyDrive/AI Projects/Opentunes AI'
ZIP_PATH = os.path.join(BASE_PATH, 'code/opentunes.zip')

if os.path.exists(ZIP_PATH):
    print(f"📦 Unzipping {ZIP_PATH}...")
    !unzip -o -q "$ZIP_PATH"
else:
    print(f"❌ Zip file not found at: {ZIP_PATH}")

# Install Requirements
!pip install fastapi uvicorn pyngrok supabase python-multipart python-dotenv loguru diffusers transformers accelerate torch torchaudio soundfile --quiet
!pip install -e .

In [ ]:
# 3. Configure & Launch
import os
from pyngrok import ngrok
from supabase import create_client
import nest_asyncio
import uvicorn
import threading
import time
from google.colab import userdata

# --- SECRETS ---
try:
    ngrok_token = userdata.get('NGROK_TOKEN')
    sb_url = userdata.get('SUPABASE_URL')
    sb_key = userdata.get('SUPABASE_SERVICE_KEY')
except Exception:
    print("⚠️ Secrets not found! Set NGROK_TOKEN, SUPABASE_URL, SUPABASE_SERVICE_KEY in Colab Secrets.")
    ngrok_token, sb_url, sb_key = "token", "url", "key"

ngrok.set_auth_token(ngrok_token)

# --- ENVIRONMENT ---
BASE_PATH = '/content/drive/MyDrive/AI Projects/Opentunes AI'
os.environ['ACE_CHECKPOINT_PATH'] = os.path.join(BASE_PATH, 'models') # Use your custom folder for models
os.environ['NEXT_PUBLIC_SUPABASE_URL'] = sb_url
os.environ['SUPABASE_SERVICE_ROLE_KEY'] = sb_key

# --- START TUNNEL ---
public_url = ngrok.connect(8000).public_url
print(f"\n🔥 API Live at: {public_url}\n")

# --- SYNC URL TO DATABASE ---
try:
    sb = create_client(sb_url, sb_key)
    sb.table('system_config').upsert({'key': 'api_url', 'value': public_url}).execute()
    print("✅ Synced URL to Supabase 'system_config'")
except Exception as e:
    print(f"❌ Failed to sync to Supabase: {e}")

# --- START APP ---
from acestep.api.main import app
nest_asyncio.apply()
uvicorn.run(app, port=8000)